Titre....

In [ ]:
!pip install gdown 

In [ ]:
import os
import gdown
import pandas as pd
import geopandas as gpd

# ----------------------------

# STOP_TIMES.TXT

# ----------------------------

os.makedirs("data/GTFS", exist_ok=True)
file_id_stop_times = "1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS"
url_stop_times = f"[https://drive.google.com/uc?id={file_id_stop_times}](https://drive.google.com/uc?id={file_id_stop_times})"
out_path_stop_times = "data/GTFS/stop_times.txt"

gdown.download(url_stop_times, out_path_stop_times, quiet=False)

# Lire le fichier CSV

stop_times = pd.read_csv(out_path_stop_times)
print("Stop_times :")
print(stop_times.head())

# ----------------------------

# POI GEOJSON

# ----------------------------

os.makedirs("data/poi", exist_ok=True)

poi_files = {
"bureaux": "1evXsbfXaeYq7pZKflvUdw6pBBbHsC6ge",
"commerce": "1cUIlDh_TmrYlkJpNssVNY53RznbZnJxB",
"logement": "17ajuRvgNHCCGTrhf8PzoNaoQTXTd91bV",
"restaurants": "1n2kXJIklAewwpzzweHuDFlrLHtk_3E6e",
"sports": "1OI3LJU2oaaSnwq2fJgK4SGgetkYWNMUZ"
}

gdfs = {}

for name, file_id in poi_files.items():
url = f"[https://drive.google.com/uc?id={file_id}](https://drive.google.com/uc?id={file_id})"
out_path = f"data/poi/poi_{name}.geojson"
gdown.download(url, out_path, quiet=False)

```
gdfs[name] = gpd.read_file(out_path)
print(f"\nPOI {name} :")
print(gdfs[name].head())
print(gdfs[name].crs)
```



In [1]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [osmnx]kx]
